In [1]:
import logging
import pandas as pd
import numpy as np

import torch as th

from clearml import Task

from de_anonymizer.de_anonymizer import DeAnonymizer
from utils import read_data_for_grader, compute_metrics, choose_data

In [41]:
# Read data from study 1
data_used = "famous_and_semi"
data_dir1 = f"textwash_data/study1/intruder_test/full_data_study.csv"

columns_to_read = ["type", "text", "file_id", "name", "got_name_truth_q2"]
raw_data1 = pd.read_csv(data_dir1, usecols=columns_to_read)

# Aggregate by file_id and calculate the rate of re-identification
data1 = (
    raw_data1.groupby(["type", "file_id", "name", "text"])
    .agg({"got_name_truth_q2": "mean"})
    .reset_index()
)
data1.rename(columns={"got_name_truth_q2": "human_rate"}, inplace=True)

# Define population to use
data1 = choose_data(data1, data_used)

In [42]:
data1

,type,file_id,name,text,human_rate
0,famous,famous_100_d_1_5.txt,mick jagger,PERSON_FIRSTNAME_5 PERSON_LASTNAME_6 is the le...,0.000000
1,famous,famous_101_d_1_6.txt,adele,PERSON_FIRSTNAME_1 is a LOCATION_1 female sing...,0.666667
2,famous,famous_102_d_1_7.txt,daniel radcliffe,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 is an LOC...,0.000000
3,famous,famous_103_d_1_5.txt,mick jagger,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 is a NUME...,0.000000
4,famous,famous_104_d_1_2.txt,ed sheeran,A ginger-haired OTHER_IDENTIFYING_ATTRIBUTE_1 ...,1.000000
...,...,...,...,...,...
1191,semifamous,semifamous_96_d_3_6.txt,malcolm dustan,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 is an ex ...,0.000000
1192,semifamous,semifamous_97_d_3_4.txt,clyde fernandes,This person is a footballler and keen to be a ...,0.000000
1193,semifamous,semifamous_98_d_3_5.txt,kenny kramm,"PERSON_FIRSTNAME_1 is a relatively small, pale...",0.000000
1194,semifamous,semifamous_99_d_3_9.txt,walter chorn,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 played th...,0.000000


In [43]:
# Read data from study 2
data_dir2 = f"textwash_data/study2/intruder_test/full_data_study.csv"

columns_to_read = ["text", "file_id", "person_long", "got_name_truth_q2_long"]
raw_data2 = pd.read_csv(data_dir2, usecols=columns_to_read)

# Aggregate by file_id and calculate the rate of re-identification
data2 = (
    raw_data2.groupby(["file_id", "person_long", "text"])
    .agg({"got_name_truth_q2_long": "mean"})
    .reset_index()
)
data2.rename(columns={"got_name_truth_q2_long": "human_rate", "person_long": "name"}, inplace=True)


In [45]:
data2["type"] = ["famous"] * len(data2)
data2

,file_id,name,text,human_rate,type
0,adele_1002.txt,adele,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 is a NUME...,0.500000,famous
1,adele_1043.txt,adele,PERSON_FIRSTNAME_1 is a singer-songwriter and ...,1.000000,famous
2,adele_1048.txt,adele,PERSON_FIRSTNAME_3 is an LOCATION_1 singer. PR...,0.500000,famous
3,adele_1051.txt,adele,PERSON_FIRSTNAME_1 is a LOCATION_1 singer who ...,0.333333,famous
4,adele_1067.txt,adele,PERSON_FIRSTNAME_2 is a LOCATION_2 singer and ...,0.000000,famous
...,...,...,...,...,...
1075,watson_916.txt,emma watson,PRONOUN is a LOCATION_1 actress and known for ...,0.000000,famous
1076,watson_921.txt,emma watson,"For me, PRONOUN is a very sexy woman with char...",0.000000,famous
1077,watson_931.txt,emma watson,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 full name...,1.000000,famous
1078,watson_937.txt,emma watson,PERSON_FIRSTNAME_1 PERSON_LASTNAME_2 is an act...,0.000000,famous


In [48]:
# combine the two dataframes
data = pd.concat([data1, data2])
data

,type,file_id,name,text,human_rate
0,famous,famous_100_d_1_5.txt,mick jagger,PERSON_FIRSTNAME_5 PERSON_LASTNAME_6 is the le...,0.000000
1,famous,famous_101_d_1_6.txt,adele,PERSON_FIRSTNAME_1 is a LOCATION_1 female sing...,0.666667
2,famous,famous_102_d_1_7.txt,daniel radcliffe,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 is an LOC...,0.000000
3,famous,famous_103_d_1_5.txt,mick jagger,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 is a NUME...,0.000000
4,famous,famous_104_d_1_2.txt,ed sheeran,A ginger-haired OTHER_IDENTIFYING_ATTRIBUTE_1 ...,1.000000
...,...,...,...,...,...
1075,famous,watson_916.txt,emma watson,PRONOUN is a LOCATION_1 actress and known for ...,0.000000
1076,famous,watson_921.txt,emma watson,"For me, PRONOUN is a very sexy woman with char...",0.000000
1077,famous,watson_931.txt,emma watson,PERSON_FIRSTNAME_1 PERSON_LASTNAME_1 full name...,1.000000
1078,famous,watson_937.txt,emma watson,PERSON_FIRSTNAME_1 PERSON_LASTNAME_2 is an act...,0.000000


In [49]:
# Split the data into training and remaining data
train_data, val_data = train_test_split(data, test_size=0.2, random_state=seed)

# Split the remaining data into validation and test data
val_data, test_data = train_test_split(val_data, test_size=0.5, random_state=seed)

NameError: name 'train_test_split' is not defined